In [1]:
# Notebook for estimating development from Rt and cases
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
#Settings for plotting
fs_label = 14

parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label
             }

plt.rcParams.update(parameters)

#locale.setlocale(locale.LC_TIME, "Danish") 

In [3]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [4]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [5]:
# dfCase = pd.read_csv(latestdir+'\\Cases_by_age.csv',delimiter=';')
dfCase = pd.DataFrame(columns=dfAdm.columns)
dfTest = pd.DataFrame(columns=dfAdm.columns)
dfPosP = pd.DataFrame(columns=dfAdm.columns)

In [6]:
# for d in curDates:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    curdf = pd.read_csv(curFilePath,delimiter=';',dtype=str)


    curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
    curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
    curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))

    dfCase.loc[curIndex] = curdf['Antal_bekræftede_COVID-19'].values
    dfTest.loc[curIndex] = curdf['Antal_testede'].values
    dfPosP.loc[curIndex] = curdf['Procent_positive'].values



In [7]:
dfCaseDiff = dfCase.diff().iloc[1:]
dfAdmDiff = dfAdm.diff().iloc[1:]
# dfAdmDiff
allCols = dfCase.columns

plotDates =  pd.to_datetime(dfCaseDiff.index,format='%d_%m_%Y')

dfCaseDiff.tail()

Aldersgruppe,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
13_04_2021,396,857,939,662,638,575,271,162,45,7,4552
20_04_2021,582,988,982,785,746,644,301,137,21,10,5196
27_04_2021,520,906,889,724,755,656,289,112,30,4,4885
04_05_2021,477,975,1204,836,827,699,372,120,16,6,5532
11_05_2021,503,1347,1416,942,926,786,377,107,26,1,6431


In [8]:
fig,ax1 = plt.subplots(1,1)

curCol = '20-29'
ax1.plot(plotDates,dfCaseDiff[curCol],'*-')


plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
tSpan = np.arange(0,14)
tSpan = np.arange(-4,14)


curData = dfCaseDiff['20-29']
# curData = dfCaseDiff['40-49']
# curData = dfCaseDiff['I alt']
iniCount = dfCaseDiff.iloc[-5,2]
# iniCount = dfCaseDiff.iloc[-5,4]
# iniCount = dfCaseDiff.iloc[-5,-1]
firstDate = np.datetime64('2021-04-13')
# iniCount = dfCaseDiff.iloc[-3,-1]
# firstDate = np.datetime64('2021-04-27')
# iniCount = dfCaseDiff.iloc[-1,-1]
# firstDate = np.datetime64('2021-05-11')
tSpanPlot = np.arange(firstDate,firstDate+np.timedelta64(7*14,'D'),np.timedelta64(7,'D'))
tSpanPlot = np.arange(firstDate-np.timedelta64(7*4,'D'),firstDate+np.timedelta64(7*14,'D'),np.timedelta64(7,'D'))


fig,ax1 = plt.subplots(1,1)

genTime = 4.7 / 7

curRT = 1.0
curCount = iniCount * (curRT**(genTime*tSpan))
curSum = curCount.sum()
ax1.plot(tSpanPlot,curCount,':',label=f'Fremskrivning med {curRT:.1f}, Total: {curSum:.0f}') 
curRT = 1.1
curCount = iniCount * (curRT**(genTime*tSpan))
curSum = curCount.sum()
ax1.plot(tSpanPlot,curCount,':',label=f'Fremskrivning med {curRT:.1f}, Total: {curSum:.0f}') 
curRT = 1.2
curCount = iniCount * (curRT**(genTime*tSpan))
curSum = curCount.sum()
ax1.plot(tSpanPlot,curCount,':',label=f'Fremskrivning med {curRT:.1f}, Total: {curSum:.0f}') 
curRT = 1.3
curCount = iniCount * (curRT**(genTime*tSpan))
curSum = curCount.sum()
ax1.plot(tSpanPlot,curCount,':',label=f'Fremskrivning med {curRT:.1f}, Total: {curSum:.0f}') 
curRT = 1.4
curCount = iniCount * (curRT**(genTime*tSpan))
curSum = curCount.sum()
ax1.plot(tSpanPlot,curCount,':',label=f'Fremskrivning med {curRT:.1f}, Total: {curSum:.0f}') 


ax1.plot(plotDates,curData,'*-')
# ax1.plot(plotDates,dfCaseDiff['I alt'],'k.--')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.set_ylim(bottom = 0,top=max(curData))
ax1.set_ylim(bottom = 0,top=iniCount*2)
# ax1.set_xlim([tSpanPlot[0],tSpanPlot[-1]])
ax1.set_xlim([tSpanPlot[0],np.datetime64('2021-06-01')])

ax1.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:


# fig,ax1 = plt.subplots(1,1)
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios': [3, 1]})

showMax = False

tStart = np.datetime64('2021-04-13')

tEndWeeks = 18
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64

xLims = [tStart-tDelta*2,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]
iniRow


curAge = '30-39'
curData = dfCaseDiff[curAge].values
iniCount = iniRow[curAge].values[0]

ax1.set_title(curAge+' årige')

genTime = 4.7 / 7 # Generation time, in weeks

# allRTs = np.arange(1.0,1.5,0.1)
allRTs = np.arange(1.0,1.3,0.1)

lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

for curRT in allRTs:
    
    modelCount = iniCount * (curRT**(genTime*tSpan))
    modelSum = modelCount.sum()
    ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','))
    ax2.plot(tSpanPlot,np.cumsum(modelCount),'.-',label=f'RT: {curRT:.1f}'.replace('.',',')+f'\nTotal: {modelSum:,.0f} indtil {lastDateStr}'.replace(',','.')) 
    # ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',',')+f' \nTotal: {modelSum:,.0f} indtil {lastDateStr}'.replace(',','.')) 


ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')

if showMax:
    dataMax = max(curData)
    ax1.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede, anden bølge')
    ax1.set_ylim(bottom = 0,top=dataMax*1.1)
else:
    ax1.set_ylim(bottom = 0,top=iniCount*2.5)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

ax1.set_xlim(xLims)
# ax1.set_xlim([tSpanPlot[0],tSpanPlot[-1]])
# ax1.set_xlim([tSpanPlot[0],np.datetime64('2021-06-01')])

ax1.set_ylabel('Ugentlige smittetilfælde')

ax2.set_yscale('log')
ax2.set_ylabel('Kummulerede smittetilfælde [logaritmisk akse]')
# ax2.grid(axis='y')  

ax1.grid()
ax2.grid()

# ax1.legend()
ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [142]:
# Vaccinationskalender
vaccKal = pd.DataFrame()

neverDate = np.datetime64('2021-09-30')
vaccKal['0-9'] = [neverDate,neverDate,neverDate]
vaccKal['10-19'] = [np.datetime64('2021-05-24'),np.datetime64('2021-06-14'),np.datetime64('2021-07-19')]
vaccKal['20-29'] = [np.datetime64('2021-06-14'),np.datetime64('2021-07-19'),np.datetime64('2021-08-31')]
vaccKal['30-39'] = [np.datetime64('2021-06-21'),np.datetime64('2021-08-01'),np.datetime64('2021-08-31')]
vaccKal['40-49'] = [np.datetime64('2021-05-24'),np.datetime64('2021-06-28'),np.datetime64('2021-08-02')]
vaccKal['50-59'] = [np.datetime64('2021-05-03'),np.datetime64('2021-05-31'),np.datetime64('2021-07-05')]
vaccKal['60-69'] = [np.datetime64('2021-04-05'),np.datetime64('2021-05-24'),np.datetime64('2021-06-28')]
vaccKal['70-79'] = [np.datetime64('2021-03-29'),np.datetime64('2021-04-26'),np.datetime64('2021-05-31')]
vaccKal['80-89'] = [np.datetime64('2021-02-01'),np.datetime64('2021-03-29'),np.datetime64('2021-04-26')]
vaccKal['90+'] =   [np.datetime64('2021-02-01'),np.datetime64('2021-02-22'),np.datetime64('2021-03-15')]
vaccKal['I alt'] = [np.datetime64('2021-07-19'),np.datetime64('2021-08-02'),np.datetime64('2021-08-31')]

vaccKal

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
0,2021-09-30,2021-05-24,2021-06-14,2021-06-21,2021-05-24,2021-05-03,2021-04-05,2021-03-29,2021-02-01,2021-02-01,2021-07-19
1,2021-09-30,2021-06-14,2021-07-19,2021-08-01,2021-06-28,2021-05-31,2021-05-24,2021-04-26,2021-03-29,2021-02-22,2021-08-02
2,2021-09-30,2021-07-19,2021-08-31,2021-08-31,2021-08-02,2021-07-05,2021-06-28,2021-05-31,2021-04-26,2021-03-15,2021-08-31


In [155]:
# Forskellige Rt for aldersgruppe, hvor mange ugentligt smittede har vi før gruppen er vaccineret

fig,ax1 = plt.subplots(1,1,figsize=(12,6))
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios': [4, 1]})

showMax = False

tStart = np.datetime64('2021-04-13')

tEndWeeks = 22
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64

xLims = [tStart-tDelta*2,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]
iniRow


curAge = '20-29'
# curAge = '40-49'
# curAge = '50-59'
# vaccIni = np.datetime64('2021-06-21')
# vaccOne = np.datetime64('2021-08-01')
# vaccTwo = np.datetime64('2021-08-31')
vaccIni = vaccKal[curAge][0]
vaccOne = vaccKal[curAge][1]
vaccTwo = vaccKal[curAge][2]

curData = dfCaseDiff[curAge].values
iniCount = iniRow[curAge].values[0]


# ax1.bar(vaccIni,max(curData),color='springgreen',label='Vaccination påbegyndes')
# ax1.bar(vaccOne,max(curData),color='limegreen',label='Første dosis afsluttet')
# ax1.bar(vaccTwo,max(curData),color='darkgreen',label='Forventet færdigvaccineret')

# ax1.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='springgreen',label='Vaccination start')
# ax1.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='limegreen',label='Første dosis')
# ax1.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='forestgreen',label='Færdigvaccineret')

ax1.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='aquamarine',label='Vaccination start')
ax1.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='palegreen',label='Første dosis')
ax1.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='springgreen',label='Færdigvaccineret')

ax1.set_title(curAge+' årige')

genTime = 4.7 / 7 # Generation time, in weeks

# allRTs = np.arange(1.0,1.5,0.1)
allRTs = np.arange(1.0,1.3,0.1)

lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

cmap = plt.cm.get_cmap('plasma',len(allRTs))
# for curRT in allRTs:
for k in range(0,len(allRTs)):
    curRT = allRTs[k]
    
    modelCount = iniCount * (curRT**(genTime*tSpan))
    modelSum = modelCount.sum()
    ax1.plot(tSpanPlot,modelCount,'.-',label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
    # ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','),color=cmap(k))


ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data (SSI)')
# ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')


if showMax:
    dataMax = max(curData)
    ax1.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede, anden bølge')
    ax1.set_ylim(bottom = 0,top=dataMax*1.1)
else:
    ax1.set_ylim(bottom = 0,top=iniCount*2.5)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

ax1.set_xlim(xLims)

ax1.set_ylabel('Ugentlige smittetilfælde')
ax1.set_xlabel('Dato')

ax1.grid(color='black')

# ax1.legend()
ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [153]:

# allAges = dfCaseDiff.columns[0:6]
allAges = dfCaseDiff.columns

    # Forskellige Rt for aldersgruppe, hvor mange ugentligt smittede har vi før gruppen er vaccineret

# fig,ax1 = plt.subplots(1,1,figsize=(12,6))
fig,allAxes = plt.subplots(len(allAges),1,sharex=True,figsize=(16,36))
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios': [4, 1]})

showMax = False

tStart = np.datetime64('2021-04-13')

tEndWeeks = 22
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64

xLims = [tStart-tDelta*2,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]
iniRow


for i in range(0,len(allAges)):
    curAx = allAxes.flatten()[i]
    curAge = allAges[i]
    # curAge = '30-39'
    # curAge = '40-49'
    # curAge = '50-59'
    # vaccIni = np.datetime64('2021-06-21')
    # vaccOne = np.datetime64('2021-08-01')
    # vaccTwo = np.datetime64('2021-08-31')
    vaccIni = vaccKal[curAge][0]
    vaccOne = vaccKal[curAge][1]
    vaccTwo = vaccKal[curAge][2]

    curData = dfCaseDiff[curAge].values
    iniCount = iniRow[curAge].values[0]


    # ax1.bar(vaccIni,max(curData),color='springgreen',label='Vaccination påbegyndes')
    # ax1.bar(vaccOne,max(curData),color='limegreen',label='Første dosis afsluttet')
    # ax1.bar(vaccTwo,max(curData),color='darkgreen',label='Forventet færdigvaccineret')


    # curAx.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='springgreen',label='Vaccination start')
    # curAx.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='limegreen',label='Første dosis')
    # curAx.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='forestgreen',label='Færdigvaccineret')
    
    curAx.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='aquamarine',label='Vaccination start')
    curAx.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='palegreen',label='Første dosis')
    curAx.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='springgreen',label='Færdigvaccineret')

    curAx.set_title(curAge+' årige')
    if (curAge == 'I alt'):
        curAx.set_title(curAge)

    genTime = 4.7 / 7 # Generation time, in weeks

    # allRTs = np.arange(1.0,1.5,0.1)
    allRTs = np.arange(1.0,1.3,0.1)

    lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

    cmap = plt.cm.get_cmap('plasma',len(allRTs))
    # for curRT in allRTs:
    for k in range(0,len(allRTs)):
        curRT = allRTs[k]
        
        modelCount = iniCount * (curRT**(genTime*tSpan))
        modelSum = modelCount.sum()
        curAx.plot(tSpanPlot,modelCount,'.-',linewidth=0.75,markersize=6,label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
        # curAx.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','),color=cmap(k))


    curAx.plot(plotDates,curData,'k*:',linewidth=0.5,markersize=10,label=f'Data (SSI)')
    # curAx.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')


    if showMax:
        dataMax = max(curData)
        curAx.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede, anden bølge')
        curAx.set_ylim(bottom = 0,top=dataMax*1.1)
    else:
        curAx.set_ylim(bottom = 0,top=iniCount*4)

    curAx.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

    curAx.set_xlim(xLims)

    curAx.set_ylabel('Ugentlige smittetilfælde')
    # curAx.set_xlabel('Dato')

    # curAx.grid(color='black')
    curAx.grid(axis='y')

    # curAx.legend()
    curAx.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/RT_Fremskrivning_Aldersgrupper')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [152]:

# allAges = dfCaseDiff.columns[0:6]
allAges = dfCaseDiff.columns

    # Forskellige Rt for aldersgruppe, hvor mange ugentligt smittede har vi før gruppen er vaccineret

# fig,ax1 = plt.subplots(1,1,figsize=(12,6))
fig,allAxes = plt.subplots(len(allAges),1,sharex=True,figsize=(16,36))
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios': [4, 1]})

showMax = False

tStart = np.datetime64('2021-04-27')

tEndWeeks = 22
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64

xLims = [tStart-tDelta*2,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]
iniRow


for i in range(0,len(allAges)):
    curAx = allAxes.flatten()[i]
    curAge = allAges[i]
    # curAge = '30-39'
    # curAge = '40-49'
    # curAge = '50-59'
    # vaccIni = np.datetime64('2021-06-21')
    # vaccOne = np.datetime64('2021-08-01')
    # vaccTwo = np.datetime64('2021-08-31')
    vaccIni = vaccKal[curAge][0]
    vaccOne = vaccKal[curAge][1]
    vaccTwo = vaccKal[curAge][2]

    curData = dfCaseDiff[curAge].values
    iniCount = iniRow[curAge].values[0]


    # ax1.bar(vaccIni,max(curData),color='springgreen',label='Vaccination påbegyndes')
    # ax1.bar(vaccOne,max(curData),color='limegreen',label='Første dosis afsluttet')
    # ax1.bar(vaccTwo,max(curData),color='darkgreen',label='Forventet færdigvaccineret')

    # curAx.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='springgreen',label='Vaccination start')
    # curAx.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='limegreen',label='Første dosis')
    # curAx.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='forestgreen',label='Færdigvaccineret')
    
    curAx.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='aquamarine',label='Vaccination start')
    curAx.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='palegreen',label='Første dosis')
    curAx.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='springgreen',label='Færdigvaccineret')

    curAx.set_title(curAge+' årige')
    if (curAge == 'I alt'):
        curAx.set_title(curAge)

    genTime = 4.7 / 7 # Generation time, in weeks

    allRTs = np.arange(1.0,1.5,0.1)
    # allRTs = np.arange(1.0,1.4,0.1)

    lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

    cmap = plt.cm.get_cmap('plasma',len(allRTs))
    # for curRT in allRTs:
    for k in range(0,len(allRTs)):
        curRT = allRTs[k]
        
        modelCount = iniCount * (curRT**(genTime*tSpan))
        modelSum = modelCount.sum()
        curAx.plot(tSpanPlot,modelCount,'.-',linewidth=0.75,markersize=6,label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
        # curAx.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','),color=cmap(k))


    curAx.plot(plotDates,curData,'k*:',linewidth=0.5,markersize=10,label=f'Data (SSI)')
    # curAx.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')


    if showMax:
        dataMax = max(curData)
        curAx.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede, anden bølge')
        curAx.set_ylim(bottom = 0,top=dataMax*1.1)
    else:
        curAx.set_ylim(bottom = 0,top=iniCount*4)

    curAx.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

    curAx.set_xlim(xLims)

    curAx.set_ylabel('Ugentlige smittetilfælde')
    # curAx.set_xlabel('Dato')

    # curAx.grid(color='black')
    curAx.grid(axis='y')

    # curAx.legend()
    curAx.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/RT_Fremskrivning_Aldersgrupper_SlutApril')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …